In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({
    'font.size': 14,
    'axes.grid': True,
    'axes.prop_cycle': plt.cycler(color=['k'])
})
import pandas as pd
from uncertainties import ufloat as uf, unumpy as unp
import uncertainties.umath as umath
from collections import OrderedDict
from pint import UnitRegistry
import pint
u = UnitRegistry()
u.load_definitions("../units.txt")
u.setup_matplotlib()
u.mpl_formatter = '{:L}'
pint.set_application_registry(u)
import pint_pandas
# M_ = u.Measurement
Q_ = u.Quantity
def M_(val, err, unit):
    if type(val) == list or type(val) == np.ndarray or type(val) == pd.Series:
        if type(err) == list or type(err) == np.ndarray:
            return Q_(np.array([uf(n, s) for (n, s) in zip(val, err)]), unit)
        else:
            return Q_(np.array([uf(n, err) for n in val]), unit)
    else:
        return u.Measurement(val, err, unit)

def mk_numpy(arr, to):
    if type(to) == str: to = u(to)
    return np.array(list(map(lambda x: x.m_as(to), arr))) * to

def np_split(arr):
    return np.array([(i.n, i.s) for i in arr.magnitude]) * arr.units

def errorbar(x, y):
    x = np_split(x)
    y = np_split(y)
    # print(x[:, 0])
    plt.errorbar(x[:, 0], y[:, 0], yerr=y[:, 1], xerr=x[:, 1])
    plt.gca().xaxis.set_units(x.units)
    plt.gca().yaxis.set_units(y.units)

def plot(x, y):
    x = np_split(x)
    y = np_split(y)
    # print(x[:, 0])
    plt.plot(x[:, 0], y[:, 0])

def polyfit(x, y):
    x = np_split(x)
    y = np_split(y)
    ((k, b), cov) = np.polyfit(x[:, 0].magnitude, y[:, 0].magnitude, deg=1, w=1/y[:, 1].magnitude, cov=True)
    err = np.sqrt(np.diag(cov))
    return (M_(k, err[0], (1 * y.units * (x.units)**-1)).to_reduced_units(), M_(b, err[1], (1 * y.units)))

def minmax(x):
    x = np_split(x)
    return mk_numpy([x[:, 0].min(), x[:, 0].max()], x.units)

def hack(x):
    return [i for i in x]

_ = mk_numpy([1*u.г], 'г') # test
_ = M_(1, 1, 'с')
_ = M_([1, 2], 1, 'с')
_ = M_([1, 2], [1, 2], 'с')

try:
    _PINT = _PINT
except:
    _PINT = False
if not _PINT:
    _PINT = True
    @pint.register_unit_format("typst")
    def format_unit_typst(unit, registry, **options):
        return " dot ".join((f'"{u}"^({p})' if p != 1 else f'"{u}"')for u, p in unit.items())
u.default_format = "typst"

def load(path):
    dfs = pd.read_excel(path, engine="odf", header=[0,1], sheet_name=None)
    return {
        sheet: pd.DataFrame({
            c[0]: pint_pandas.PintArray(df[c[0]].to_numpy().flatten(), c[1]) if str(c[1]) != '1' else df[c[0]].to_numpy().flatten()
            # if str(c[1]) != '1' else df[c[0]].to_numpy().flatten()
            for c in df.columns
        }) for (sheet, df) in dfs.items()
    }

def make_cols(df, cols):
    from itertools import zip_longest
    import warnings
    c0 = df.columns.get_level_values(0)
    if len(cols) > len(c0):
        warnings.warn("Too many columns")
        cols = cols[:len(c0)]
    try:
        c1 = df.columns.get_level_values(1)
    except:
        c1 = ['']*len(c0)
    df.columns = pd.MultiIndex.from_tuples(zip(c0, [col if col else cc1 for col, cc1 in zip_longest(cols, c1)]))

PA_ = pint_pandas.PintArray

def save():
    _temptype = type(M_(1.0, 0.1, "м"))
    _temptype1 = type(Q_(1.0, "м"))
    import inspect
    import toml
    import warnings
    class MyEncoder(toml.TomlNumpyEncoder):
        def __init__(self, _dict=dict, preserve=False):
            import uncertainties
            import pint
            super(MyEncoder, self).__init__(_dict, preserve)
            self.dump_funcs[uncertainties.core.Variable] = self._dump_ufloat
            self.dump_funcs[uncertainties.core.AffineScalarFunc] = self._dump_ufloat
            self.dump_funcs[pint.facets.measurement.objects.Measurement] = self._dump_pint
            self.dump_funcs[_temptype] = self._dump_pint
            self.dump_funcs[_temptype1] = self._dump_pint
            self.dump_funcs[pd.DataFrame] = self._dump_pandas
        def _dump_ufloat(self, v):
            return self.dump_inline_table({"n": v.n, "s": v.s, "r": str(v).replace("+/-", "±")})
        def _dump_pint(self, v):
            n, s = str(v.magnitude).split("+/-")
            u = f'{v.units:typst}'
            return self.dump_inline_table({
                "n": f'${n}space{u}$',
                "s": f'${s}space{u}$',
                "r": f'${n} plus.minus {s}space{u}$'
            })
        def _dump_pandas(self, v):
            cs = v.columns.to_frame().to_numpy()
            if cs.shape[1] == 3:
                return self.dump_inline_table({
                    "cols": [f"${c1}, {c2}$" if c2 != "No Unit" else f"${c1}$" for c1, c2 in zip(v.columns.get_level_values(1), v.columns.get_level_values(2))],
                    "vals": v.values
                })
            if cs.shape[1] == 2:
                return self.dump_inline_table({
                    "cols": v.columns.get_level_values(1),
                    "vals": v.values
                })
            warnings.warn("Unknown column index shape")
            

    with open('data.toml', 'w') as _f:
        _vars = %who_ls
        _d = {
            k: v
            for k in _vars
            if not inspect.ismodule((v := globals()[k]))
            and not inspect.isfunction(v)
            and not k.startswith('_')
            and not repr(v).startswith('<')
            and not (type(v) == pd.DataFrame and type(v.columns) != pd.MultiIndex)
            or type(v) == pint.facets.measurement.objects.Measurement
            or type(v) == _temptype
            # or type(v) == _temptype1
        }
        toml.dump(_d, _f, encoder=MyEncoder())